In [1]:
if 'google.colab' in str(get_ipython()):
    import sys
    from google.colab import drive, output
    drive.mount('/content/drive')
    !pip install torchaudio
    !pip install wandb --upgrade
    output.clear()
    print("Running on colab")
    %load_ext autoreload
    %autoreload 1
    %cd '/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project'
    root = "/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project"
else:
    print("Running local")
    root = "/home/mirp_ai/Documents/Daniel_Nieto/PhD/AutoEncoders_Ecoacoustics"
    root_path = "/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido"

Running local


In [2]:
import os
import numpy as np

import datetime
from datetime import timedelta

import torch
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from torchvision.utils import make_grid
from Jaguas_DataLoader_rainless import SoundscapeData

import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = os.listdir(path_to_dir)
    return sorted([ filename for filename in filenames if filename.endswith( suffix ) ])

def find_wav_filenames( path_to_dir, suffix=".wav" ):
    filenames = os.listdir(path_to_dir)
    return sorted([ filename for filename in filenames if filename.endswith( suffix ) ])

In [4]:
files_Ais = find_csv_filenames(f"{root}/Zamuro/AI_Caract")
file_rain = f"{root}/Zamuro/Complementary_Files/Result_rain_merged.xlsx"
files_udas = f"{root}/Zamuro/Complementary_Files/UDAS Pasivo_20221001_Zamuro.xlsx"
files_covers = f"{root}/Zamuro/Complementary_Files/ReyZamuro_softLabels.csv"
files_recorders = os.listdir("/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/")
path_recorders = "/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/"

In [5]:
UDAS = pd.read_excel(f"{files_udas}", sheet_name="Template")
covers = pd.read_csv(f"{files_covers}")
# AIs = pd.read_csv(files_Ais)
habitat = ["bosque"]* 2 + ["pastizal"] + ["bosque"] + ["borde"] * 2 + ["bosque"] +  ["parche"] *2 + ["bosque"]*4 + ["borde"]*11 + ["bosque"] * 9 + ["parche"]*2 + ["pastizal"]*4 + ["parche"]*2 + ["sabana"]*10 + ["borde"] + ["sabana"] + ["pastizal"] + ["sabana"] + ["bosque"]*3 + ["pastizal"]*2 + ["potrero"] * 2 + ["bosque"] + ["borde"]* 5 +["bosque"] * 9 + ["sabana"]* 2 + ["pastizal"] + ["parche"]
habitat = [
    "bosque", "bosque", "pastizal", "borde", "borde", "borde", "bosque", "sabana", "bosque", "borde",
    "bosque", "bosque", "bosque", "bosque", "bosque", "borde", "borde", "pastizal", "potrero", "sabana",
    "borde", "bosque", "bosque", "sabana", "borde", "bosque", "borde", "bosque", "bosque", "bosque", "borde",
    "potrero", "bosque", "bosque", "sabana", "bosque", "bosque", "sabana", "parche", "bosque", "pastizal",
    "bosque", "bosque", "borde", "bosque", "borde", "borde", "bosque", "parche", "bosque", "bosque", "sabana",
    "bosque", "pastizal", "pastizal", "sabana", "sabana", "borde", "bosque", "sabana", "bosque", "sabana",
    "bosque", "sabana", "borde", "borde", "borde", "borde", "pastizal", "bosque", "pastizal", "pastizal",
    "bosque", "bosque", "bosque", "bosque", "sabana", "pastizal", "parche", "borde", "sabana", "sabana", "parche",
    "parche", "sabana", "borde", "sabana", "parche", "bosque", "sabana", "parche", "bosque", "sabana", "borde",
    "borde", "sabana", "sabana", "borde", "bosque", "bosque", "bosque", "bosque", "bosque", "bosque", "bosque", "bosque"
]

keep_cols = ["field_number_PR", "longitud_IG", "latitude_IG", "day_init_TI", "day_end_TI", "samp_rate_RE"]

In [6]:
UDAS = UDAS[keep_cols]
UDAS["habitat"] = habitat
UDAS.info()
covers.rename(columns={"field_numb":"field_number_PR", "1":"pasture", "2": "savanna", "3": "forest"}, inplace=True)
a=UDAS
b=covers
merged_df = a.merge(b, on='field_number_PR', how='left')
result_df = merged_df[merged_df['pasture'].notna()]
result_df.reset_index(inplace=True)
result_df.drop("index", axis=1, inplace=True)
result_df["cover"] = result_df[["pasture", "savanna", "forest"]].idxmax(axis=1)
result_df.to_csv(f"{root}//Zamuro/Complementary_Files/zamuro_recorders.csv")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   field_number_PR  106 non-null    object 
 1   longitud_IG      106 non-null    float64
 2   latitude_IG      106 non-null    float64
 3   day_init_TI      106 non-null    int64  
 4   day_end_TI       106 non-null    int64  
 5   samp_rate_RE     106 non-null    int64  
 6   habitat          106 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 5.9+ KB


/tmp/ipykernel_174837/3762597280.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df.drop("index", axis=1, inplace=True)
/tmp/ipykernel_174837/3762597280.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df["cover"] = result_df[["pasture", "savanna", "forest"]].idxmax(axis=1)


In [7]:
audios = pd.DataFrame(columns=result_df.columns)
total_files=0
for i in range(len(result_df)):
    aux = pd.DataFrame(columns=result_df.columns)
    path = root_path+"/"+result_df.loc[i, "field_number_PR"]
    files = os.listdir(path)
    total_files+=len(files)
    aux["Filename"] = files
    name=[i.split('.', 1)[0] for i in files]
    aux["name_FI"] = name
    for col in aux.columns[0:-2]:
        aux[col] = result_df.loc[i,col]
    audios = pd.concat([audios, aux])
audios.to_csv(f"{root}/Zamuro/Complementary_Files/zamuro_audios.csv")
audios.reset_index(inplace=True)

In [9]:
rain = pd.read_excel(file_rain)
rain = rain[["field_number_PR", "name_FI","rain_FI"]]
audios["rain_FI"] = "-"
count = 0
for i in range(len(audios)):
    if i%10000 == 0:
        print(i)
    try:
        audio = audios.loc[i]
        audios.loc[i, "rain_FI"] = rain.loc[rain["field_number_PR"] == audio["field_number_PR"]].loc[rain["name_FI"] == audio["Filename"]]["rain_FI"].values[0]
    except:
        count+=1
audios.to_csv(f"{root}//Zamuro/Complementary_Files/zamuro_audios.csv")

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
